In [ ]:
import numpy as np, pandas as pd, lightgbm as lgbm, catboost as cb, xgboost as xg
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")
sub = pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")
lb = LabelEncoder()
#Speed things up
train = train.sample(frac=0.1)
print(len(train))
import gc

In [ ]:
import sklearn.utils, time
import matplotlib.pyplot as plt
from IPython.display import clear_output
def plot_classifiers(X,y):
    estimators = sklearn.utils.all_estimators(type_filter=["classifier"])
    estimators = estimators + [('lgbm', lgbm.LGBMClassifier), ('xgb', xg.XGBClassifier)] #('cat', cb.CatBoostClassifier), it too slow
    rv,nrv = [],[]
    for est in estimators:
        try:
            t = time.monotonic()
            gc.collect()
            scores = cross_val_score(est[1](), X,y, cv=5)
            v=(est,scores, time.monotonic() - t)
            if np.isnan(scores[0]):
                nrv.append(v)
            else:
                rv.append(v)
            clear_output(wait=True)
            fig = plt.figure()
            rv.sort(key = lambda x:np.mean(x[1]), reverse = True)
            trv = rv+nrv
            plt.figure(figsize=(20,4))
            plt.bar([s[0][0] for s in rv],[s[2] for s in rv], color = 'r', width = 0.4, align='edge', label ='Time')
            plt.xticks(rotation=90)
            plt.twinx()
            plt.bar([s[0][0] for s in rv], [np.nanmean(s[1]) for s in rv], color = 'b', width = 0.4, label = 'Score')
            fig.legend(loc="upper right")
            plt.show()
            _ = [print(x[0][0],np.round(np.nanmean(x[1]),4), np.round(x[2],1), "s") for x in rv[0:5]]
        except KeyboardInterrupt:
            raise KeyboardInterrupt()
        except:
            print(est, "Exception")
    return trv



In [ ]:
rv = plot_classifiers(train.iloc[:,1:-1],  lb.fit_transform(train['target']))